# Scraping What one month looks like from Insider

## 2.1 Contents<a id='2.1_Contents'></a>
* [2.1 Importing Relevant Tools](#2.1_Importing)
* [2.2 Defining the Request](#2.2_URL)
* [2.3 Grab | Date](#2.4_scrape_date)
* [2.4 Grab | Header](#2.4_scrape_header)
* [2.5 Grab | Content](#2.4_scrape_content)
* [2.6 Clean | Send to DataFrame](#2.5_review)
* [2.7 Save](#2.6_save)


**Importing Relevant Tools**<a id='2.1_Importing'></a>

In [25]:
import json
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
import csv
import re
import pickle

%reload_ext watermark

In [26]:
#the below needs to be reviewed for all websites; notably the time format

from datetime import date
from datetime import datetime
today = date.today()
d = today.strftime("%m-%d-%y")

For reference.

In [27]:
%watermark -d -t -v -p pandas

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

pandas: 1.1.3



**Defining the Request**<a id='2.2_URL'></a>

In [28]:
url = 'https://www.businessinsider.com/hong-kong-china-national-security-law-one-month-what-changed-2020-7'
content = requests.get(url).content
url

'https://www.businessinsider.com/hong-kong-china-national-security-law-one-month-what-changed-2020-7'

In [29]:
soup = soup(content,'html.parser')
# bsobj

**Grab | Date**<a id='2.4_scrape_date'></a>

In [30]:
for date in soup.findAll('section',{'class':'byline-wrapper col-12'}):
    print(date.text.strip())

Bill Bostock


                    2020-08-02T07:10:00Z
                  



 



Facebook Icon
The letter F.

 

Email icon
An envelope. It indicates the ability to send an email.





 




Twitter icon
A stylized bird with an open mouth, tweeting.


Twitter
 



LinkedIn icon
The word "in".


LinkedIn
 



Fliboard icon
A stylized letter F.


Flipboard
 



Link icon
An image of a chain link. It symobilizes a website link url.


Copy Link


The date is in there but needs to be cleaned.

**Grab | Header**<a id='2.4_scrape_header'></a>

In [31]:
for title in soup.findAll('h1'):
    print(title.text.strip())

Teenage arrests, blank protest signs, and a key election postponed: What one month of China's new national security law for Hong Kong has looked like


In [32]:
title

<h1 class="post-headline">Teenage arrests, blank protest signs, and a key election postponed: What one month of China's new national security law for Hong Kong has looked like</h1>

In [33]:
TAG_RE = re.compile(r'<[^>]+>')

In [34]:
def remove_tags(text):
    return TAG_RE.sub('', text)

In [35]:
remove_tags(str(title))

"Teenage arrests, blank protest signs, and a key election postponed: What one month of China's new national security law for Hong Kong has looked like"

Confirmed that's the title; a little longer of a journey to get it.

**Grab | Content**<a id='2.4_scrape_content'></a>

In [36]:
for bodies in soup.find_all('div',{'class','content-lock-content'}):
    print(bodies.text.strip())

Thursday, July 30, marked one month since China imposed a new national-security law on Hong Kong.A lot has happened in a very short time.Vocal activists fled to safety abroad, political organizations disbanded, and pro-democracy candidates were banned from running in September's legislative election (which was also postponed on Friday).But pro-democracy activists have vowed to fight on, with some finding new ways to protest without falling afoul of the law, and others planning a parliament in exile.Elsewhere, China and Britain have exchanged barbs after the UK promised to give 3 million Hong Kongers a path to citizenship.Visit Business Insider's homepage for more stories.

Thursday, July 30, marked one month since China's new national security law came into force in Hong Kong.From June 30, China has wielded the power to define and punish "separatism, subversion, terrorism and foreign interference" in the city as it sees fit. This means that anti-China sentiment — be it waving banners, 

In [37]:
len(bodies)

90

Confirmed that's the bottom; needs internal work.

**Clean | Send to DataFrame**<a id='2.5_review'></a>

First the date from string to a datetime object.

In [60]:
for date in soup.findAll('section',{'class':'byline-wrapper col-12'}):
    date.text.strip()

In [61]:
date

<section class="byline-wrapper col-12">
<div class="byline" data-e2e-name="byline">
<div class="byline-content" data-e2e-name="byline-content">
<div class="byline-row">
<div class="byline-author-container">
<div class="byline-author headline-bold">
<span class="" data-e2e-name="Bill Bostock">
<a class="byline-link byline-author-name" data-e2e-name="byline-author-name" href="https://www.businessinsider.com/author/bill-bostock">Bill Bostock</a></span>
</div>
<div class="byline-timestamp headline-regular js-date-format" data-e2e-name="byline-timestamp" data-timestamp="2020-08-02T07:10:00Z">
                    2020-08-02T07:10:00Z
                  </div>
</div>
</div>
</div>
</div> <section aria-label="Share This Post" class="share-wrapper share-wrapper-top" data-e2e-name="share-wrapper-top" data-share-top="" data-track-page-area="Post Top">
<div class="group-icons" data-group-icons="">
<a aria-label="Click to visit us on Facebook" class="share-link facebook" data-e2e-name="share-link-fa

In [62]:
date = remove_tags(str(date))
date

'\n\n\n\n\n\n\nBill Bostock\n\n\n                    2020-08-02T07:10:00Z\n                  \n\n\n\n \n\n\n\nFacebook Icon\nThe letter F.\n\n \n\nEmail icon\nAn envelope. It indicates the ability to send an email.\n\n\n\n\n\n \n\n\n\n\nTwitter icon\nA stylized bird with an open mouth, tweeting.\n\n\nTwitter\n \n\n\n\nLinkedIn icon\nThe word "in".\n\n\nLinkedIn\n \n\n\n\nFliboard icon\nA stylized letter F.\n\n\nFlipboard\n \n\n\n\nLink icon\nAn image of a chain link. It symobilizes a website link url.\n\n\nCopy Link\n \n\n\n'

In [65]:
date_list = date.split('\n')
date_list = date_list[10]
date_str = re.sub(' ','',date_list)
date_list = date_str.split('T')
day_pub = datetime.strptime(date_list[0], '%Y-%m-%d').date()
day_pub

datetime.date(2020, 8, 2)

In [66]:
df_date = pd.DataFrame([day_pub])

In [67]:
type(df_date)

pandas.core.frame.DataFrame

In [68]:
df_date

,0
0,2020-08-02


Now the title.

In [69]:
for title in soup.findAll('h1'):
    print(title.text.strip())

Teenage arrests, blank protest signs, and a key election postponed: What one month of China's new national security law for Hong Kong has looked like


In [70]:
title

<h1 class="post-headline">Teenage arrests, blank protest signs, and a key election postponed: What one month of China's new national security law for Hong Kong has looked like</h1>

In [71]:
title = remove_tags(str(title))

In [72]:
title

"Teenage arrests, blank protest signs, and a key election postponed: What one month of China's new national security law for Hong Kong has looked like"

In [73]:
df_title = pd.DataFrame([title])

In [74]:
df_title

,0
0,"Teenage arrests, blank protest signs, and a ke..."


In [75]:
type(df_title)

pandas.core.frame.DataFrame

These items are manually added.

In [76]:
country = 'US'
df_country = pd.DataFrame([country])
source = 'Insider'
df_source = pd.DataFrame([source])
file_name = 'insider_5'
df_file_name = pd.DataFrame([file_name])


Finally, the news.

In [77]:
news1 = []
for bodies in soup.find_all('div',{'class','content-lock-content'}):
    news1.append(bodies.text.strip())

In [78]:
news1

['Thursday, July 30, marked one month since China imposed a new national-security law on Hong Kong.A lot has happened in a very short time.Vocal activists fled to safety abroad, political organizations disbanded, and pro-democracy candidates were banned from running in September\'s legislative election (which was also postponed on Friday).But pro-democracy activists have vowed to fight on, with some finding new ways to protest without falling afoul of the law, and others planning a parliament in exile.Elsewhere, China and Britain have exchanged barbs after the UK promised to give 3 million Hong Kongers a path to citizenship.Visit Business Insider\'s homepage for more stories.\n\nThursday, July 30, marked one month since China\'s new national security law came into force in Hong Kong.From June 30, China has wielded the power to define and punish "separatism, subversion, terrorism and foreign interference" in the city as it sees fit.\xa0This means that anti-China sentiment — be it waving

In [79]:
df_news = pd.DataFrame()

In [80]:
df_news['article_body'] = news1

In [81]:
df_news.head(2)

,article_body
0,"Thursday, July 30, marked one month since Chin..."


In [82]:
df_news['article_body'] = df_news.article_body.str.cat(sep=' ')

In [83]:
df_news = df_news.article_body[0]

In [84]:
df_news = df_news.replace(r'\\?','')

In [85]:
df_news = pd.DataFrame([df_news])

In [86]:
type(df_news)

pandas.core.frame.DataFrame

In [87]:
df_news.columns = ['Article']

In [88]:
df_news.head()

,Article
0,"Thursday, July 30, marked one month since Chin..."


**Bringing it together.**<a id='2.5_bit'></a>

In [89]:
df_5_insider = pd.concat([df_file_name,df_date,df_source,df_country,df_title,df_news],axis = 1, ignore_index=False)

In [92]:
df_5_insider.columns = ['file_name','date','source','country','title','article']

In [93]:
df_5_insider.head()

,file_name,date,source,country,title,article
0,insider_5,2020-08-02,Insider,US,"Teenage arrests, blank protest signs, and a ke...","Thursday, July 30, marked one month since Chin..."


**Saving**<a id='2.6_save'></a>

In [94]:
cd

C:\Users\rands


Saving it to Excel.

In [95]:
# df = pd.DataFrame(reviewlist)

# index=False below so that we don't get the dataframe index on the side; we just use the excel index
df_5_insider.to_csv('./_Capstone_Two_NLP/data/_news/insider_5.csv', index=False)

print('Complete')

Complete
